In [182]:
import folium
import folium.plugins
import pymongo
import pandas as pd
from folium.features import *
from math import sin, cos, sqrt, atan2, radians

In [183]:
pd.options.display.max_rows = 999

In [184]:
client = pymongo.MongoClient("localhost", 27017)
coll = client.col_en_datos.cities

In [185]:
#db.getCollection('cities').find({
#   population: {$gt:10000},
#   location: {
#     $near: {
#       $geometry: {
#          type: "Point" ,
#          coordinates: [ -74.0721 , 4.7110 ]
#       }
#     }
#   }
#})

In [186]:
def get_nearest_city(coor):
    lat = coor[1]
    lon = coor[0]
    res = coll.find({
        'population': {'$gt':5000},
        'location': {
            '$near': {
                '$geometry': {
                    'type': "Point" ,
                    'coordinates': [ lon , lat ]
                }
            }
        }
    })
    return res[1]

In [187]:
def get_distance(p0, p1):
    R = 6373.0

    lat1 = radians(p0[1])
    lon1 = radians(p0[0])
    lat2 = radians(p1[1])
    lon2 = radians(p1[0])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

In [188]:
cities = coll.find({'population': {'$gt':5000}, 'country':'co'})
#cities = coll.find({'population': {'$gt':10000}})

In [189]:
distances_df = []
for city in cities:
    nearest_city = get_nearest_city(city['location']['coordinates'])
    d = {}
    d['city'] = city['city']
    d['nearest_city'] = nearest_city['city']
    d['distance'] = get_distance(city['location']['coordinates'], nearest_city['location']['coordinates'])
    distances_df.append(d)
distances_df = pd.DataFrame(distances_df)

In [193]:
distances_df.sort_values('distance').tail(5)

,city,distance,nearest_city
38,Bajo Baudó,79.000410,Istmina
362,San Andrés,152.082628,Corn Island
312,Puerto Leguízamo,170.122551,Cartagena del Chairá
189,Inírida,202.808878,Puerto Ayacucho
246,Mitú,314.528032,San José del Guaviare


In [191]:
class DivIcon(MacroElement):
    def __init__(self, html='', size=(30,30), anchor=(0,0), style=''):
        """TODO : docstring here"""
        super(DivIcon, self).__init__()
        self._name = 'DivIcon'
        self.size = size
        self.anchor = anchor
        self.html = html
        self.style = style

        self._template = Template(u"""
            {% macro header(this, kwargs) %}
              <style>
                .{{this.get_name()}} {
                    {{this.style}}
                    }
              </style>
            {% endmacro %}
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.divIcon({
                    className: '{{this.get_name()}}',
                    iconSize: [{{ this.size[0] }},{{ this.size[1] }}],
                    iconAnchor: [{{ this.anchor[0] }},{{ this.anchor[1] }}],
                    html : "{{this.html}}",
                    });
                {{this._parent.get_name()}}.setIcon({{this.get_name()}});
            {% endmacro %}
            """)

In [192]:
m = folium.Map([4.7110, -74.0721],
                tiles="Mapbox Bright",
                zoom_start=5)

folium.map.Marker(
    [4.7110, -74.0721]).add_to(m)

folium.map.Marker(
    [4.7110, -74.0721],
    icon=DivIcon(
        size=(150,36),
        anchor=(150,0),
        html='Bogotá',
        style="""
            font-size:36px;
            background-color: transparent;
            border-color: transparent;
            text-align: right;
            """
        )
    ).add_to(m)


m